## Deploy Mphasis DeepInsights Named Entity Recognizer Model Package from AWS Marketplace 


Mphasis DeepInsights Named Entity Recognizer is an efficient way of identifying named entities present in the corpus of text. This solution applies NLP techniques to extract the named entities which can be used for further text analytics and for providing useful insights about the text. The model takes a corpus of text as input file, processes it and provides a csv file containing all the named entities present in the text as output.

This sample notebook shows you how to deploy Mphasis DeepInsights Named Entity Recognizer using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to Mphasis DeepInsights Named Entity Recognizer. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

#### Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Output Result](#D.-Output-Result)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page Mphasis DeepInsights Named Entity Recognizer
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [1]:
model_package_arn='arn:aws:sagemaker:us-east-2:786796469737:model-package/mphasis-deepinsights-named-entity-recognition'

In [2]:
import pandas as pd
import json
import os
import boto3
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
from IPython.display import Image
from PIL import Image as ImageEdit

In [7]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket=sagemaker_session.default_bucket()
bucket

'sagemaker-us-east-2-786796469737'

### 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [8]:
model_name='named-entity-recognizer'

content_type='text/csv'

real_time_inference_instance_type='ml.m5.xlarge'
batch_transform_inference_instance_type='ml.m5.large'

#### A. Create an endpoint

In [14]:
def predict_wrapper(endpoint, session):
    return sage.RealTimePredictor(endpoint, session,content_type=content_type)

#create a deployable model from the model package.

model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

In [ ]:
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

Once endpoint has been created, you would be able to perform real-time inference.

#### B. Create input payload

##### Input Instructions

•	The input must be a text file with 'utf-8' encoding.

•	Maximum input file sizes : 350kb ( ~ 2500 rows of text )


In [22]:
file_path = os.getcwd()
file_name = 'data/input/Input.txt'

with open(file_name,'r') as file:
    input_text = file.read().replace('\n', '')

In [23]:
input_text

'The World Health Organization is a specialized agency of the United Nations responsible for international public health. It is part of the United Nations Sustainable Development Group. It is headquartered in Geneva, Switzerland, with six semi-autonomous regional offices and 150 field offices worldwide. The WHO was established in 7 April 1948. The current Director-General is Tedros Adhanom, former health minister and foreign minister of Ethiopia, who began his five-year term on 1 July 2017.'

In [24]:
output_file_name = 'data/output/Named_Entity_sample_data.csv'

#### C. Perform real-time inference

In [25]:
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name $model_name \
    --body fileb://$file_name \
    --content-type $content_type \
    --region $sagemaker_session.boto_region_name \
    $output_file_name

{
    "ContentType": "text/csv; charset=utf-8",
    "InvokedProductionVariant": "AllTraffic"
}


#### D. Output Result

The processed output is of the form .csv file with following columns :
    
    Entity: This column will contain the original entity from the text data.
    
    Entity_Label: This columns will contain the named entity recognized with corresponding entity.
    
    index: This columns will contain number representing the order in which entity appears in the text data.

In [ ]:
output_df  = pd.read_csv("data/output/Named_Entity_sample_data.csv")
output_df  = output_df.drop('Unnamed: 0',axis = 1)

In [27]:
print("Output: ")
output_df

Output: 


,index,Entity,Entity_Label
0,9,Tedros Adhanom,PERSON
1,0,The World Health Organization,ORG
2,1,the United Nations,ORG
3,2,the United Nations Sustainable Development Group,ORG
4,7,WHO,ORG
5,3,Geneva,GPE
6,4,Switzerland,GPE
7,10,Ethiopia,GPE
8,8,7 April 1948,DATE
9,11,five-year,DATE


#### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [28]:
predictor=sage.predictor.Predictor(model_name, sagemaker_session,content_type)
predictor.delete_endpoint(delete_endpoint_config=True)

### 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [ ]:
import json 
import uuid


transformer = model.transformer(1, 'ml.m5.large')
transformer.transform('s3://mphasis-marketplace/mphasis-named-entity/Input.txt', content_type='text/csv')
transformer.wait()
#transformer.output_path
print("Batch Transform complete")
bucketFolder = transformer.output_path.rsplit('/')[3]

In [31]:
#print(s3bucket,s3prefix)
s3_conn = boto3.client("s3")
bucket_name="sagemaker-us-east-2-786796469737"
with open('Named_Entity_sample_data', 'wb') as f:
    s3_conn.download_fileobj(bucket_name, bucketFolder+'/Input.txt.out', f)
    print("Output file loaded from bucket")

Output file loaded from bucket


In [32]:
output_df  = pd.read_csv("data/output/Named_Entity_sample_data.csv")
output_df  = output_df.drop('Unnamed: 0',axis = 1)

In [33]:
print("Output: ")
output_df

Output: 


,index,Entity,Entity_Label
0,9,Tedros Adhanom,PERSON
1,0,The World Health Organization,ORG
2,1,the United Nations,ORG
3,2,the United Nations Sustainable Development Group,ORG
4,7,WHO,ORG
5,3,Geneva,GPE
6,4,Switzerland,GPE
7,10,Ethiopia,GPE
8,8,7 April 1948,DATE
9,11,five-year,DATE


### 4. Clean-up

#### A. Delete the model

In [34]:
model.delete_model()

#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.